In [1]:
#!/home/was966/micromamba/envs/responder/bin/python
#sbatch --mem 64G -c 4 -t 100:00:00 -p gpu_quad --gres=gpu:a100:1 ./loco_pft.py

import sys

sys.path.insert(0, '/home/was966/Research/mims-conceptor/')
from conceptor.utils import plot_embed_with_label
from conceptor import PreTrainer, FineTuner, loadconceptor #, get_minmal_epoch
from conceptor.utils import plot_embed_with_label,plot_performance, score2
from conceptor.tokenizer import CANCER_CODE, CONCEPT

import os
from tqdm import tqdm
from itertools import chain
import pandas as pd
import numpy as np
import random, torch
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'white', font_scale=1.3)
import warnings
warnings.filterwarnings("ignore")

def onehot(S):
    assert type(S) == pd.Series, 'Input type should be pd.Series'
    dfd = pd.get_dummies(S, dummy_na=True)
    nanidx = dfd[dfd[np.nan].astype(bool)].index
    dfd.loc[nanidx, :] = np.nan
    dfd = dfd.drop(columns=[np.nan])*1.
    cols = dfd.sum().sort_values(ascending=False).index.tolist()
    dfd = dfd[cols]
    return dfd


data_path = '/home/was966/Research/mims-conceptor/paper/00_data/'
df_label = pd.read_pickle(os.path.join(data_path, 'ITRP.PATIENT.TABLE'))
df_tpm = pd.read_pickle(os.path.join(data_path, 'ITRP.TPM.TABLE'))
df_tpm.shape, df_label.shape

dfcx = df_label.cancer_type.map(CANCER_CODE).to_frame('cancer_code').join(df_tpm)

df_task = onehot(df_label.response_label)
size = df_label.groupby('cohort').size()
size = size.index + "\n(n = " + size.astype(str) + ")"
cohorts = df_label.groupby('cohort').size().sort_values().index.tolist()
#cohorts = ['Choueiri']

In [2]:
def _connect(x):
    from itertools import chain
    return list(set(chain(*[i.split(':') for i in x])))
    
concept_genes = CONCEPT.groupby('BroadCelltypePathway').Genes.apply(lambda x:x.tolist()).apply(_connect)

In [3]:
from baseline.immnue_score.scorer import ssGSEA, avgAbundance

In [4]:
res=[]
for name, g_list in tqdm(concept_genes.items()):
    avg = avgAbundance(g_list, name)
    score = avg.fit_transform(df_tpm)
    res.append(score)

43it [00:18,  2.35it/s]


In [7]:
dfc = pd.concat(res,axis=1)
dfc.columns = dfc.columns.map(lambda x:x.replace('NAG_', ''))
dfc = dfc[CONCEPT.groupby('BroadCelltypePathway').Concept_index.mean().sort_values().index]
dfc.to_csv('02_avgAbd_readouts_celltype.csv')

In [8]:
geneset_genes = CONCEPT.Genes.apply(lambda x:x.split(':'))
res=[]
for name, g_list in tqdm(geneset_genes.items()):
    avg = avgAbundance(g_list, name)
    score = avg.fit_transform(df_tpm)
    res.append(score)

dfg = pd.concat(res,axis=1)
dfg.columns = dfg.columns.map(lambda x:x.replace('NAG_', ''))
dfg.to_csv('01_avgAbd_readouts_geneset.csv')

132it [00:53,  2.45it/s]
